In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from datetime import datetime,timedelta

import os

os.chdir('Python')
from past_features import *
from elo_features import *
from categorical_features import *
from utilities import *
from additional_stuff import *
os.chdir('../')
import numpy as np
import pandas as pd

# Preprocessing

## Load data
The initial data was downloaded from [http://tennis-data.co.uk/alldata.php](http://tennis-data.co.uk/alldata.php) and stored in data folder. (Further work could be to automate this)

At first load the data from the files and put them together in a dataframe

In [16]:
import glob
#filenames=list(glob.glob("../Data/20*.xls*"))
filenames=list(glob.glob("Data/20*.xls*"))
l = [pd.read_excel(filename,encoding='latin-1') for filename in filenames]
data=pd.concat(l,0,sort=False)

data = data[list(data.columns)[:13]+["Wsets","Lsets","Comment"] +["PSW","PSL","B365W","B365L"]]

## Data Cleaning (#1)

In [17]:
### Data cleaning
data=data.sort_values("Date")
data["WRank"]=data["WRank"].replace(np.nan,0)
data["WRank"]=data["WRank"].replace("NR",2000)
data["LRank"]=data["LRank"].replace(np.nan,0)
data["LRank"]=data["LRank"].replace("NR",2000)
data["WRank"]=data["WRank"].astype(int)
data["LRank"]=data["LRank"].astype(int)
data["Wsets"]=data["Wsets"].astype(float)
data["Lsets"]=data["Lsets"].replace("`1",1)
data["Lsets"]=data["Lsets"].astype(float)

data['Winner']=data['Winner'].str.strip()
data['Loser']=data['Loser'].str.strip()

# the dict 'names_to_correct' is in Python/additional_stuff.py
data = data.replace(names_to_correct)

data=data.reset_index(drop=True)

### Storage of the raw dataset
#data.to_csv("../Generated Data/atp_data_raw.csv",index=False)
#dump(data,"../Generated Data/atp_data_raw")
dump(data,"Generated Data/atp_data_raw")

## Preproc. Elo-Rating

The elo-score / elo-rating is a value that describes the performance in the past of the player. It takes into account the wins and losses in the past but also the performance of the opponent. So a success against a strong comtetitor has a larger gain as against a weak opponent. Further information on [Wiki](https://en.wikipedia.org/wiki/Elo_rating_system)

The calculation-method of the elo-score is not unique and the algorithm is empirical with variats. Here is a rather simple implementation of the calculation. The *K-factor* e.g. is set constant to 32, and could be seen as *hyer-parameter*. (Working on that is for the future) 

The function is stored in [Python/elo_features.py](Python/elo_features.py)

It figured out that this feature belongs to the most important ones in this project.

(TODO: Create also Surface-dependant elo-scores for each underground)

In [18]:
### Elo rankings data

# Computing of the elo ranking of each player at the beginning of each match.
elo_rankings = compute_elo_rankings(data)
data = pd.concat([data,elo_rankings],1)

### Storage of the dataset with elo
#data.to_csv("../Generated Data/atp_data.csv",index=False)
#dump(data,"../Generated Data/atp_data")
dump(data,"Generated Data/atp_data")

Elo rankings computing...
5000 matches computed...
10000 matches computed...
15000 matches computed...
20000 matches computed...
25000 matches computed...
30000 matches computed...
35000 matches computed...
40000 matches computed...
45000 matches computed...
50000 matches computed...


In [6]:
#######################################################
# Until here basic with elo                ############
#######################################################

In [28]:
# Loading data if necessary

#data = pd.read_csv("../Generated Data/atp_data.csv")
#data.Date = data.Date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
#data = load("../Generated Data/atp_data")
data = load("Generated Data/atp_data")

Just for visual checking if data distribution is ok. Loading data sometimes gives strange datasets. 
An indication that something is wrong is when a year has only few matches (<1000). If something is wrong latest the splitting function will give errors.


In [ ]:
data.groupby(pd.Grouper(key='Date',freq='Y')).agg('count')

# Feature engineering

## Selecting data to proceed
For further procedure not all data is necessary. Of course it depends on how much we want to train on. 
The earlier data was primaly needed to calculate the elo-scores. Even we start here in 2008 there will remain more than 30k matches.

In [21]:
#beg = datetime.datetime(2008,1,1) 
#end = data.Date.iloc[-1]
#indices = data[(data.Date>beg)&(data.Date<=end)].index

beg_year = 2008
indices = data[(data.Date.dt.year >= beg_year)].index

## Calculate the other features
Most features look in the past of the player and his performance. 
For the moment the most features analyze the past **150** and **300** days. 
I don't know if larger time spans may cause better results. The time span(s) is a kind of a hyperparameter.

The functions below for the feature calculation is stored in [Python/past_features.py](Python/past_features.py)

Note: There are cases where nothing is to say about the past. That means that *missing values / NaN* are produced in the features. Some classifieres can handle them (XGBoost) others not (AdaBoost with DecisionTree e.g.). These missing values will be filtered out later after splitting.

(TODO: These functions take their time. It would be nice to run them in parallel to accelerate the building. I tried and gave up. It seems that it is not as easy to use *multiprocessing* or similar in an iPython enviroment. It would have to be outsourced to a py-file and run in a shell.)

In [22]:
features_player      = features_past_generation(features_player_creation,5,"playerft5",data,indices)

features_duo         = features_past_generation(features_duo_creation,150,"duoft",data,indices)
features_duo_300     = features_past_generation(features_duo_creation,300,"duoft_300",data,indices)

features_general     = features_past_generation(features_general_creation,150,"generalft",data,indices)
features_general_300 = features_past_generation(features_general_creation,300,"generalft_300",data,indices)

features_recent      = features_past_generation(features_recent_creation,150,"recentft",data,indices)
features_recent_300  = features_past_generation(features_recent_creation,300,"recentft_300",data,indices)


0/31748 matches treated.
100/31748 matches treated.
200/31748 matches treated.
300/31748 matches treated.
400/31748 matches treated.
500/31748 matches treated.
600/31748 matches treated.
700/31748 matches treated.
800/31748 matches treated.
900/31748 matches treated.
1000/31748 matches treated.
1100/31748 matches treated.
1200/31748 matches treated.
1300/31748 matches treated.
1400/31748 matches treated.
1500/31748 matches treated.
1600/31748 matches treated.
1700/31748 matches treated.
1800/31748 matches treated.
1900/31748 matches treated.
2000/31748 matches treated.
2100/31748 matches treated.
2200/31748 matches treated.
2300/31748 matches treated.
2400/31748 matches treated.
2500/31748 matches treated.
2600/31748 matches treated.
2700/31748 matches treated.
2800/31748 matches treated.
2900/31748 matches treated.
3000/31748 matches treated.
3100/31748 matches treated.
3200/31748 matches treated.
3300/31748 matches treated.
3400/31748 matches treated.
3500/31748 matches treated.
3600

28700/31748 matches treated.
28800/31748 matches treated.
28900/31748 matches treated.
29000/31748 matches treated.
29100/31748 matches treated.
29200/31748 matches treated.
29300/31748 matches treated.
29400/31748 matches treated.
29500/31748 matches treated.
29600/31748 matches treated.
29700/31748 matches treated.
29800/31748 matches treated.
29900/31748 matches treated.
30000/31748 matches treated.
30100/31748 matches treated.
30200/31748 matches treated.
30300/31748 matches treated.
30400/31748 matches treated.
30500/31748 matches treated.
30600/31748 matches treated.
30700/31748 matches treated.
30800/31748 matches treated.
30900/31748 matches treated.
31000/31748 matches treated.
31100/31748 matches treated.
31200/31748 matches treated.
31300/31748 matches treated.
31400/31748 matches treated.
31500/31748 matches treated.
31600/31748 matches treated.
31700/31748 matches treated.
0/31748 matches treated.
100/31748 matches treated.
200/31748 matches treated.
300/31748 matches trea

25600/31748 matches treated.
25700/31748 matches treated.
25800/31748 matches treated.
25900/31748 matches treated.
26000/31748 matches treated.
26100/31748 matches treated.
26200/31748 matches treated.
26300/31748 matches treated.
26400/31748 matches treated.
26500/31748 matches treated.
26600/31748 matches treated.
26700/31748 matches treated.
26800/31748 matches treated.
26900/31748 matches treated.
27000/31748 matches treated.
27100/31748 matches treated.
27200/31748 matches treated.
27300/31748 matches treated.
27400/31748 matches treated.
27500/31748 matches treated.
27600/31748 matches treated.
27700/31748 matches treated.
27800/31748 matches treated.
27900/31748 matches treated.
28000/31748 matches treated.
28100/31748 matches treated.
28200/31748 matches treated.
28300/31748 matches treated.
28400/31748 matches treated.
28500/31748 matches treated.
28600/31748 matches treated.
28700/31748 matches treated.
28800/31748 matches treated.
28900/31748 matches treated.
29000/31748 ma

22500/31748 matches treated.
22600/31748 matches treated.
22700/31748 matches treated.
22800/31748 matches treated.
22900/31748 matches treated.
23000/31748 matches treated.
23100/31748 matches treated.
23200/31748 matches treated.
23300/31748 matches treated.
23400/31748 matches treated.
23500/31748 matches treated.
23600/31748 matches treated.
23700/31748 matches treated.
23800/31748 matches treated.
23900/31748 matches treated.
24000/31748 matches treated.
24100/31748 matches treated.
24200/31748 matches treated.
24300/31748 matches treated.
24400/31748 matches treated.
24500/31748 matches treated.
24600/31748 matches treated.
24700/31748 matches treated.
24800/31748 matches treated.
24900/31748 matches treated.
25000/31748 matches treated.
25100/31748 matches treated.
25200/31748 matches treated.
25300/31748 matches treated.
25400/31748 matches treated.
25500/31748 matches treated.
25600/31748 matches treated.
25700/31748 matches treated.
25800/31748 matches treated.
25900/31748 ma

19400/31748 matches treated.
19500/31748 matches treated.
19600/31748 matches treated.
19700/31748 matches treated.
19800/31748 matches treated.
19900/31748 matches treated.
20000/31748 matches treated.
20100/31748 matches treated.
20200/31748 matches treated.
20300/31748 matches treated.
20400/31748 matches treated.
20500/31748 matches treated.
20600/31748 matches treated.
20700/31748 matches treated.
20800/31748 matches treated.
20900/31748 matches treated.
21000/31748 matches treated.
21100/31748 matches treated.
21200/31748 matches treated.
21300/31748 matches treated.
21400/31748 matches treated.
21500/31748 matches treated.
21600/31748 matches treated.
21700/31748 matches treated.
21800/31748 matches treated.
21900/31748 matches treated.
22000/31748 matches treated.
22100/31748 matches treated.
22200/31748 matches treated.
22300/31748 matches treated.
22400/31748 matches treated.
22500/31748 matches treated.
22600/31748 matches treated.
22700/31748 matches treated.
22800/31748 ma

16300/31748 matches treated.
16400/31748 matches treated.
16500/31748 matches treated.
16600/31748 matches treated.
16700/31748 matches treated.
16800/31748 matches treated.
16900/31748 matches treated.
17000/31748 matches treated.
17100/31748 matches treated.
17200/31748 matches treated.
17300/31748 matches treated.
17400/31748 matches treated.
17500/31748 matches treated.
17600/31748 matches treated.
17700/31748 matches treated.
17800/31748 matches treated.
17900/31748 matches treated.
18000/31748 matches treated.
18100/31748 matches treated.
18200/31748 matches treated.
18300/31748 matches treated.
18400/31748 matches treated.
18500/31748 matches treated.
18600/31748 matches treated.
18700/31748 matches treated.
18800/31748 matches treated.
18900/31748 matches treated.
19000/31748 matches treated.
19100/31748 matches treated.
19200/31748 matches treated.
19300/31748 matches treated.
19400/31748 matches treated.
19500/31748 matches treated.
19600/31748 matches treated.
19700/31748 ma

13200/31748 matches treated.
13300/31748 matches treated.
13400/31748 matches treated.
13500/31748 matches treated.
13600/31748 matches treated.
13700/31748 matches treated.
13800/31748 matches treated.
13900/31748 matches treated.
14000/31748 matches treated.
14100/31748 matches treated.
14200/31748 matches treated.
14300/31748 matches treated.
14400/31748 matches treated.
14500/31748 matches treated.
14600/31748 matches treated.
14700/31748 matches treated.
14800/31748 matches treated.
14900/31748 matches treated.
15000/31748 matches treated.
15100/31748 matches treated.
15200/31748 matches treated.
15300/31748 matches treated.
15400/31748 matches treated.
15500/31748 matches treated.
15600/31748 matches treated.
15700/31748 matches treated.
15800/31748 matches treated.
15900/31748 matches treated.
16000/31748 matches treated.
16100/31748 matches treated.
16200/31748 matches treated.
16300/31748 matches treated.
16400/31748 matches treated.
16500/31748 matches treated.
16600/31748 ma

10100/31748 matches treated.
10200/31748 matches treated.
10300/31748 matches treated.
10400/31748 matches treated.
10500/31748 matches treated.
10600/31748 matches treated.
10700/31748 matches treated.
10800/31748 matches treated.
10900/31748 matches treated.
11000/31748 matches treated.
11100/31748 matches treated.
11200/31748 matches treated.
11300/31748 matches treated.
11400/31748 matches treated.
11500/31748 matches treated.
11600/31748 matches treated.
11700/31748 matches treated.
11800/31748 matches treated.
11900/31748 matches treated.
12000/31748 matches treated.
12100/31748 matches treated.
12200/31748 matches treated.
12300/31748 matches treated.
12400/31748 matches treated.
12500/31748 matches treated.
12600/31748 matches treated.
12700/31748 matches treated.
12800/31748 matches treated.
12900/31748 matches treated.
13000/31748 matches treated.
13100/31748 matches treated.
13200/31748 matches treated.
13300/31748 matches treated.
13400/31748 matches treated.
13500/31748 ma

In [23]:
# Putting together the features of 150 and 300 days
features_recent = pd.concat([features_recent,features_recent_300], sort=False, axis=1)
features_general = pd.concat([features_general,features_general_300], sort=False, axis=1)
features_duo = pd.concat([features_duo,features_duo_300], sort=False, axis=1)

In [24]:
# save the features
dump(features_player,"Generated Data/player_features")
dump(features_duo,"Generated Data/duo_features")
dump(features_general,"Generated Data/general_features")
dump(features_recent,"Generated Data/recent_features")

In [2]:
'''
# load features to go ahead
features_player=load("Generated Data/player_features")
features_duo=load("Generated Data/duo_features")
features_general=load("Generated Data/general_features")
features_recent=load("Generated Data/recent_features")
'''

In [28]:
#features_general.shape
features_recent.shape

(63496, 14)

In [25]:
data = data.iloc[indices,:].reset_index(drop=True)
odds = data[["PSW","PSL"]]

## Encoding of categorical features

In [26]:
#features_categorical = data[["Series","Court","Surface","Round","Best of","Tournament"]]
features_categorical = data[["Series","Court","Surface","Round","Best of"]]
features_categorical_encoded = categorical_features_encoding(features_categorical)
players_encoded = features_players_encoding(data)
tournaments_encoded = features_tournaments_encoding(data)
features_onehot = pd.concat([features_categorical_encoded,players_encoded,tournaments_encoded],1)

## Duplication of rows
For the moment we have one row per match. 
We "duplicate" each row to have one row for each outcome of each match. 
Of course it isn't a simple duplication of  each row, we need to "invert" some features

In [29]:
# Elo data
elo_rankings = data[["elo_winner","elo_loser","proba_elo"]]
elo_1 = elo_rankings
elo_2 = elo_1[["elo_loser","elo_winner","proba_elo"]]
elo_2.columns = ["elo_winner","elo_loser","proba_elo"]
elo_2.proba_elo = 1-elo_2.proba_elo
elo_2.index = range(1,2*len(elo_1),2)
elo_1.index = range(0,2*len(elo_1),2)
features_elo_ranking = pd.concat([elo_1,elo_2]).sort_index(kind='merge')

In [30]:
# Categorical features
features_onehot = pd.DataFrame(np.repeat(features_onehot.values,2, axis=0),columns=features_onehot.columns)

## Odd features
We will put in for the moment. Due to the enormous influence of the odds on the results we will drop them for some models later to make our own opinion.

In [31]:
# odds feature
features_odds = pd.Series(odds.values.flatten(),name="odds")
features_odds = pd.DataFrame(features_odds)

## Putting all together

In [32]:
### Building of the final dataset
# You can remove some features to see the effect on the ROI
features = pd.concat([features_odds,
                  features_elo_ranking,
                  features_onehot,
                  features_player,
                  features_duo,
                  features_general,
                  features_recent],1)

#features.to_csv("../Generated Data/atp_data_features.csv",index=True)
dump(data,"Generated Data/atp_data_features")

In [33]:
# - Have fun with the model building ! ;-)